In [1]:
from line_solver import *
import numpy as np
GlobalConstants.set_verbose(VerboseLevel.STD)

In [2]:
model = Network('model')
# Create models
node = np.empty(4, dtype = object)
node[0] = ClassSwitch(model, 'CS', [[0, 1], [1, 0]])
node[1] = Queue(model, 'Queue1', SchedStrategy.PS)
node[2] = Queue(model, 'Queue2', SchedStrategy.PS)
node[3] = Queue(model, 'Delay', SchedStrategy.INF)

# Create job classes
jobclass = np.empty(2, dtype = object)
jobclass[0] = ClosedClass(model, 'Class1', 15, node[3], 0)
jobclass[1] = ClosedClass(model, 'Class2', 5, node[3], 0)

# Set service times
node[1].set_service(jobclass[0], Exp.fitMean(1.5))  #mean = 1.5
node[1].set_service(jobclass[1], Erlang.fitMeanAndOrder(1.5, 2))  #mean = 1.5

node[2].set_service(jobclass[0], Erlang.fitMeanAndOrder(1.5, 2))  #mean = 1.5
node[2].set_service(jobclass[1], Exp.fitMean(1.5))  #mean = 1.5

node[3].set_service(jobclass[0], Exp.fitMean(1.0))  #mean = 1
node[3].set_service(jobclass[1], Exp.fitMean(1.0))  #mean = 1

# Add links
model.add_link(node[1], node[0])
model.add_link(node[2], node[0])
model.add_link(node[0], node[3])
model.add_link(node[3], node[1])
model.add_link(node[3], node[2])

# Set routing strategies
node[0].set_routing(jobclass[0], RoutingStrategy.RAND)
node[1].set_routing(jobclass[0], RoutingStrategy.RAND)
node[2].set_routing(jobclass[0], RoutingStrategy.RAND)
node[3].set_routing(jobclass[0], RoutingStrategy.RROBIN)

node[0].set_routing(jobclass[1], RoutingStrategy.RAND)
node[1].set_routing(jobclass[1], RoutingStrategy.RAND)
node[2].set_routing(jobclass[1], RoutingStrategy.RAND)
node[3].set_routing(jobclass[1], RoutingStrategy.WRROBIN, node[1], 1)
node[3].set_routing(jobclass[1], RoutingStrategy.WRROBIN, node[2], 2)

In [3]:
# Solve with SolverJMT
simoptions = SolverJMT.defaultOptions()
simoptions.verbose = True
simoptions.seed = 23000
solver = np.array([], dtype = object)
solver = np.append(solver, SolverJMT(model, simoptions))

AvgTable = np.empty(len(solver), dtype = object)
for s in range(len(solver)):
    print(f'\nSOLVER: {solver[s].get_name()}')
    AvgTable[s] = solver[s].get_avg_table()


SOLVER: SolverJMT
JMT Model: /tmp/workspace/jsim/11635613029177468032/jmodel.jsim
JMT [method: default, lang: java, env: 21.0.8] completed in 1.000450s.
  Station JobClass    QLen    Util    RespT  ResidT    ArvR    Tput
0  Queue1   Class1  0.8618  0.4264   3.0767  0.7692  0.2857  0.2854
1  Queue1   Class2  0.5490  0.2855   2.8913  0.7228  0.1920  0.1926
2  Queue2   Class1  7.4570  0.4282  26.1672  6.5418  0.2852  0.2854
3  Queue2   Class2  9.9248  0.5718  26.0750  6.5188  0.3787  0.3789
4   Delay   Class1  0.5635  0.5635   0.9954  0.4977  0.5678  0.5697
5   Delay   Class2  0.5857  0.5857   1.0176  0.5088  0.5746  0.5745
